In [3]:
import pandas as pd
import numpy as np

Task one:
***
# Part 1: Assemble the project cohort
The project goal is to identify patients seen for drug overdose, determine if they had an active opioid at the start of the encounter, and if they had any readmissions for drug overdose.

Your task is to assemble the study cohort by identifying encounters that meet the following criteria:

1. The patient’s visit is an encounter for drug overdose
2. The hospital encounter occurs after July 15, 1999
3. The patient’s age at time of encounter is between 18 and 35 (Patient is considered to be 35 until turning 36)
***
Let's start by looking at how this data is organized in our datasets.

In [16]:
pd.read_csv("datasets/allergies.csv").head(3)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,1983-07-12,NaN,7341aae3-7606-456e-b3c1-f67d1bc6b19b,8ea228c8-f853-431e-969b-c7085602d2c1,300913006,Shellfish allergy
1,2016-10-22,NaN,54e3e6c7-2089-4158-83ef-a1d549bf3523,f70cb7eb-6986-486c-9b43-9377c549b396,419474003,Allergy to mould
2,2016-10-22,NaN,54e3e6c7-2089-4158-83ef-a1d549bf3523,f70cb7eb-6986-486c-9b43-9377c549b396,232350006,House dust mite allergy


In [15]:
pd.read_csv("datasets/encounters.csv").head(3)

,Id,START,STOP,PATIENT,PROVIDER,ENCOUNTERCLASS,CODE,DESCRIPTION,COST,REASONCODE,REASONDESCRIPTION
0,2590963f-222b-4870-b8c7-86d07f4d44b3,1959-06-18 09:43:49,1959-06-25 21:37:31,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,fa6006c2-c562-3278-859b-96b266fc3ea1,ambulatory,424441002.0,Prenatal initial visit,105.37,72892002.0,Normal pregnancy
1,ba6137e3-c8cd-42e6-9a02-eafa13d1cfa9,2008-08-29 19:14:19,2008-09-07 13:33:07,553b00b2-347c-48ec-90ee-f59924ff50eb,fe803a6f-c717-36e8-a338-9b3cdb9ee724,outpatient,698314001.0,Consultation for treatment,105.37,NaN,NaN
2,10aa7678-5c84-4ce9-81af-09272307c1f0,1999-07-12 03:05:41,1999-07-20 11:18:24,7f4ea9fb-f436-411e-ab34-e94750edfa93,cc6a930c-727f-3999-b655-179e57b30538,wellness,185349003.0,Encounter for check up (procedure),105.37,NaN,NaN


In [17]:
pd.read_csv("datasets/medications.csv").head(3)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,COST,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,1997-09-20,NaN,2043c57c-1085-45a3-87e8-a223d5b6693d,3710b40a-bcc0-4c0f-a204-e47a5f3aec98,316049,Hydromorphone 325 MG,214.96,265,56964.40,59621000.0,Pain Relief
1,1957-04-19,NaN,50f799aa-740c-4da9-ab85-49d5cbf4046b,64b4efb0-26ba-4c7f-91c1-64d8f62e49e3,1049630,diphenhydrAMINE Hydrochloride 25 MG Oral Tablet,6.53,757,4943.21,NaN,NaN
2,2008-08-29,2009-08-24,553b00b2-347c-48ec-90ee-f59924ff50eb,ba6137e3-c8cd-42e6-9a02-eafa13d1cfa9,749785,Ortho Tri-Cyclen 28 Day Pack,24.12,12,289.44,NaN,NaN


In [18]:
pd.read_csv("datasets/patients.csv").head(3)

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,ZIP
0,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,1943-03-11,NaN,999-86-7250,S99939389,X3970685X,Mrs.,Allyn942,Kreiger457,NaN,Bartoletti50,M,asian,asian_indian,F,Muhlenberg Pennsylvania US,372 Marks Heights Unit 20,Middle Paxton,Pennsylvania,NaN
1,7f4ea9fb-f436-411e-ab34-e94750edfa93,1980-09-28,NaN,999-90-4314,S99920355,X28211313X,Mr.,Kieth891,King743,NaN,NaN,M,white,irish,M,Overfield Pennsylvania US,428 Strosin Fort Suite 49,Limerick,Pennsylvania,NaN
2,553b00b2-347c-48ec-90ee-f59924ff50eb,1973-02-22,NaN,999-78-9189,S99980820,X9988931X,Mrs.,Domenica436,Fadel536,NaN,Labadie908,M,white,scottish,F,Wilkins Pennsylvania US,519 Ziemann Trail,Washington,Pennsylvania,15301.0


In [19]:
pd.read_csv("datasets/procedures.csv").head(3)

,DATE,PATIENT.x,ENCOUNTER,CODE.x,DESCRIPTION.x,COST.x,REASONCODE.x,REASONDESCRIPTION.x
0,2009-09-17,553b00b2-347c-48ec-90ee-f59924ff50eb,d08babad-74ed-4490-8830-24af7310cac7,76601001,Intramuscular injection,1858.49,NaN,NaN
1,2009-12-17,553b00b2-347c-48ec-90ee-f59924ff50eb,fbfd3904-6135-4572-a181-cb4b64878ee4,76601001,Intramuscular injection,2972.12,NaN,NaN
2,2010-03-18,553b00b2-347c-48ec-90ee-f59924ff50eb,07d30215-3df2-4a12-8e04-dfd6c04024df,76601001,Intramuscular injection,2406.65,NaN,NaN


Okay, we are primarily interested in the `encounters` table, and basically want to filter it based on the specifications given in the task.

1. `The patient’s visit is an encounter for drug overdose`

By cross-referencing the `REASONCODE` and `REASONDESCRIPTION` columns in the `encounters` dataset, we can determine that a drug overdose encounter's reason code is `55680006`. Let's import some relevant columns from `encounters` and use `55680006` to filter out non-drug overdose encounters. Then let's drop the `REASONCODE` column as it will not be needed in the future.

(Peaking ahead, I can see that our final output .csv requires specific naming conventions for its columns, so that is why I renamed some columns immediately after import.)

In [22]:
encounters = pd.read_csv("datasets/encounters.csv",
                         usecols=['PATIENT','Id','START','STOP','REASONCODE'],
                         parse_dates=['START','STOP'])

encounters.rename(columns=({'PATIENT': 'PATIENT_ID', 'Id': 'ENCOUNTER_ID',
                            'START': 'HOSPITAL_ENCOUNTER_DATE', 'STOP': 'STOP',
                            'REASONCODE': 'REASONCODE'}), inplace=True)

encounters.set_index('ENCOUNTER_ID', inplace=True)
encounters.drop(encounters[encounters.REASONCODE != 55680006].index, inplace=True)
encounters.drop('REASONCODE', axis=1, inplace=True)
encounters

,HOSPITAL_ENCOUNTER_DATE,STOP,PATIENT_ID
ENCOUNTER_ID,,,
2a917920-2701-49f0-9340-827320eef76d,2003-03-31 21:50:51,2003-04-08 13:20:43,708b81c9-21a9-411a-aae4-052ef8925b97
22874b3d-0873-40e4-a455-897d51d593f7,2012-02-18 21:50:51,2012-02-28 21:12:17,708b81c9-21a9-411a-aae4-052ef8925b97
134c5ee3-1b72-4e3e-a44a-3efebe77603b,2013-08-03 21:50:51,2013-08-13 07:44:52,708b81c9-21a9-411a-aae4-052ef8925b97
6125f147-72d4-48a4-9813-2a9dc878f055,2014-12-08 21:50:51,2014-12-17 12:25:27,708b81c9-21a9-411a-aae4-052ef8925b97
f837dcf8-af7d-43b8-b5c2-773d494617ab,2015-08-31 21:50:51,2015-09-08 12:04:08,708b81c9-21a9-411a-aae4-052ef8925b97
...,...,...,...
79c38e3f-ba11-4bab-bcb2-64984b877406,2010-10-02 18:47:24,2010-10-15 03:40:46,b464ba67-cb98-4067-8df2-6f6f1d9d5929
e63dfe7a-e889-402a-a3d8-a904dcd8ecbd,2014-12-02 18:47:24,2014-12-08 16:40:12,b464ba67-cb98-4067-8df2-6f6f1d9d5929
d90af98f-6fce-4b8b-9187-ed9621dde55c,2015-01-20 18:47:24,2015-01-26 08:37:33,b464ba67-cb98-4067-8df2-6f6f1d9d5929


Cool, time for step 2.

2. `The hospital encounter occurs after July 15, 1999`

In `encounters`, we can use `START` (now named `HOSPITAL_ENCOUNTER_DATE`) and `STOP` to figure out when an encounter occurred. There might be some edge cases where an encounter both started before and ended after 7/15/1999, but for the purposes of this exercise, we’ll interpret this requirement to mean the **entire** encounter took place after July 15, 1999.

In [23]:
encounters.drop(encounters[encounters.HOSPITAL_ENCOUNTER_DATE < pd.Timestamp(
    1999, 7, 15)].index, inplace=True)
encounters

,HOSPITAL_ENCOUNTER_DATE,STOP,PATIENT_ID
ENCOUNTER_ID,,,
2a917920-2701-49f0-9340-827320eef76d,2003-03-31 21:50:51,2003-04-08 13:20:43,708b81c9-21a9-411a-aae4-052ef8925b97
22874b3d-0873-40e4-a455-897d51d593f7,2012-02-18 21:50:51,2012-02-28 21:12:17,708b81c9-21a9-411a-aae4-052ef8925b97
134c5ee3-1b72-4e3e-a44a-3efebe77603b,2013-08-03 21:50:51,2013-08-13 07:44:52,708b81c9-21a9-411a-aae4-052ef8925b97
6125f147-72d4-48a4-9813-2a9dc878f055,2014-12-08 21:50:51,2014-12-17 12:25:27,708b81c9-21a9-411a-aae4-052ef8925b97
f837dcf8-af7d-43b8-b5c2-773d494617ab,2015-08-31 21:50:51,2015-09-08 12:04:08,708b81c9-21a9-411a-aae4-052ef8925b97
...,...,...,...
79c38e3f-ba11-4bab-bcb2-64984b877406,2010-10-02 18:47:24,2010-10-15 03:40:46,b464ba67-cb98-4067-8df2-6f6f1d9d5929
e63dfe7a-e889-402a-a3d8-a904dcd8ecbd,2014-12-02 18:47:24,2014-12-08 16:40:12,b464ba67-cb98-4067-8df2-6f6f1d9d5929
d90af98f-6fce-4b8b-9187-ed9621dde55c,2015-01-20 18:47:24,2015-01-26 08:37:33,b464ba67-cb98-4067-8df2-6f6f1d9d5929


In step 3, we are tasked with the following:

3. `The patient’s age at time of encounter is between 18 and 35 (Patient is considered to be 35 until turning 36)`

Let's import the patient dataset and immediately drop all patients that do not currently exist in our encounters dataframe so that we can prevent needless operations on irrelevant patient data.

In [25]:
patients = pd.read_csv("datasets/patients.csv",
                        usecols=['Id', 'BIRTHDATE', 'DEATHDATE'],
                        parse_dates=['BIRTHDATE', 'DEATHDATE'],
                        index_col='Id')

patients = patients[patients.index.isin(encounters.PATIENT_ID)]
patients

,BIRTHDATE,DEATHDATE
Id,,
708b81c9-21a9-411a-aae4-052ef8925b97,1986-10-28,NaT
722270fe-f8dd-46e6-aace-756450258911,1955-04-05,2016-04-21
a23ee3ee-1f6f-4efb-bc84-b8a64e240b68,1954-06-23,NaT
65b093e4-b353-447a-b9ae-290567f1c950,1995-12-03,NaT
13319bd6-49ac-4aa2-a645-5db89283d3fd,1955-04-15,2011-11-09
...,...,...
94cd65a5-ddf9-4646-b803-bd04d67e57ff,1972-03-26,NaT
4b1d8e8c-8208-4593-bc04-6ed3c7d3919d,1912-04-25,NaT
019f8618-de1f-44e4-8d4a-056fe837c5e0,1962-05-17,2016-10-06


From here, we can calculate a patient's age by subtracting their `BIRTHDATE` from the `STOP` of a given encounter. Let's create a new column `AGE_AT_VISIT` to store this data, then use it to filter out encounters where the patient's age is not in our desired range.

In [27]:
birthdays = patients.BIRTHDATE[encounters.PATIENT_ID]

encounters['AGE_AT_VISIT'] = (
    encounters['STOP'] - birthdays.values) / np.timedelta64(1, 'Y')

# Patient is considered to be 35 until turning 36
encounters.drop(encounters.loc[(encounters.AGE_AT_VISIT < 18) | (
    encounters.AGE_AT_VISIT >= 36)].index, inplace=True)

encounters

,HOSPITAL_ENCOUNTER_DATE,STOP,PATIENT_ID,AGE_AT_VISIT
ENCOUNTER_ID,,,,
22874b3d-0873-40e4-a455-897d51d593f7,2012-02-18 21:50:51,2012-02-28 21:12:17,708b81c9-21a9-411a-aae4-052ef8925b97,25.339010
134c5ee3-1b72-4e3e-a44a-3efebe77603b,2013-08-03 21:50:51,2013-08-13 07:44:52,708b81c9-21a9-411a-aae4-052ef8925b97,26.794042
6125f147-72d4-48a4-9813-2a9dc878f055,2014-12-08 21:50:51,2014-12-17 12:25:27,708b81c9-21a9-411a-aae4-052ef8925b97,28.138888
f837dcf8-af7d-43b8-b5c2-773d494617ab,2015-08-31 21:50:51,2015-09-08 12:04:08,708b81c9-21a9-411a-aae4-052ef8925b97,28.864392
010594a6-a6ff-4875-9a9e-6b213367cde0,2018-12-10 00:59:35,2018-12-20 22:57:39,65b093e4-b353-447a-b9ae-290567f1c950,23.050321
...,...,...,...,...
09512352-7f84-4960-85be-9325aa91c023,2006-03-03 00:54:04,2006-03-08 06:12:14,12842850-ab85-4655-92e1-f8c5f1ac5cc2,24.294157
13459d81-c8e0-44ba-915f-b3cc7a32c118,2017-04-20 00:54:04,2017-04-30 09:28:08,12842850-ab85-4655-92e1-f8c5f1ac5cc2,35.440549
454afa58-c029-463e-8722-d795f1383150,2009-07-21 14:36:02,2009-07-29 02:22:25,5849b818-b6b3-4ed5-b3be-19d2f6fc2aa6,31.872246


Let's move on to part 2.

***

# Part 2: Create additional fields
With your drug overdose encounter, create the following indicators:

1. `DEATH_AT_VISIT_IND`: 1 if patient died during the drug overdose encounter, 0 if the patient died at a different time
2. `COUNT_CURRENT_MEDS`: Count of active medications at the start of the drug overdose encounter
3. `CURRENT_OPIOID_IND`: 1 if the patient had at least one active medication at the start of the overdose encounter that is on the Opioids List (provided below), 0 if not
4. `READMISSION_90_DAY_IND`: 1 if the visit resulted in a subsequent drug overdose readmission within 90 days, 0 if not
5. `READMISSION_30_DAY_IND`: 1 if the visit resulted in a subsequent drug overdose readmission within 30 days, 0 if not overdose encounter, 0 if not
6. `FIRST_READMISSION_DATE`: The date of the index visit’s first readmission for drug overdose. Field should be left as N/A if no readmission for drug overdose within 90 days

***

Opioids List:
* Hydromorphone 325Mg 
* Fentanyl – 100 MCG 
* Oxycodone-acetaminophen 100 Ml`

***
For `DEATH_AT_VISIT_IND`, let's compare a patient's `DEATHDATE` to their encounter's `STOP` date. If a patient died during a given encounter, `STOP` >= `DEATHDATE`, then let's set their death indicator to `1`. 

(We will set defaults `NaN` values before we create our output file, so let's just leave them as they are for now.)

In [29]:
deathdays = (patients.loc[:, 'DEATHDATE'])[encounters.PATIENT_ID].values
death_ind_true = encounters.STOP.values >= deathdays
encounters.loc[death_ind_true, 'DEATH_AT_VISIT_IND'] = 1
encounters

,HOSPITAL_ENCOUNTER_DATE,STOP,PATIENT_ID,AGE_AT_VISIT,DEATH_AT_VISIT_IND
ENCOUNTER_ID,,,,,
22874b3d-0873-40e4-a455-897d51d593f7,2012-02-18 21:50:51,2012-02-28 21:12:17,708b81c9-21a9-411a-aae4-052ef8925b97,25.339010,NaN
134c5ee3-1b72-4e3e-a44a-3efebe77603b,2013-08-03 21:50:51,2013-08-13 07:44:52,708b81c9-21a9-411a-aae4-052ef8925b97,26.794042,NaN
6125f147-72d4-48a4-9813-2a9dc878f055,2014-12-08 21:50:51,2014-12-17 12:25:27,708b81c9-21a9-411a-aae4-052ef8925b97,28.138888,NaN
f837dcf8-af7d-43b8-b5c2-773d494617ab,2015-08-31 21:50:51,2015-09-08 12:04:08,708b81c9-21a9-411a-aae4-052ef8925b97,28.864392,NaN
010594a6-a6ff-4875-9a9e-6b213367cde0,2018-12-10 00:59:35,2018-12-20 22:57:39,65b093e4-b353-447a-b9ae-290567f1c950,23.050321,NaN
...,...,...,...,...,...
09512352-7f84-4960-85be-9325aa91c023,2006-03-03 00:54:04,2006-03-08 06:12:14,12842850-ab85-4655-92e1-f8c5f1ac5cc2,24.294157,NaN
13459d81-c8e0-44ba-915f-b3cc7a32c118,2017-04-20 00:54:04,2017-04-30 09:28:08,12842850-ab85-4655-92e1-f8c5f1ac5cc2,35.440549,NaN
454afa58-c029-463e-8722-d795f1383150,2009-07-21 14:36:02,2009-07-29 02:22:25,5849b818-b6b3-4ed5-b3be-19d2f6fc2aa6,31.872246,NaN


Let's move on to `COUNT_CURRENT_MEDS` and `CURRENT_OPIOID_IND`.

First, we need import our medicines dataset and filter out the data for patients that are not in our encounters data frame.

In [30]:
medicines = pd.read_csv("datasets/medications.csv",
                        usecols=['START', 'STOP', 'PATIENT', 'DESCRIPTION'],
                        parse_dates=['START', 'STOP'],)

medicines = medicines[medicines.PATIENT.isin(encounters.PATIENT_ID)]
medicines

,START,STOP,PATIENT,DESCRIPTION
77,2009-09-10,2010-09-05,708b81c9-21a9-411a-aae4-052ef8925b97,Seasonique 91 Day Pack
78,2011-08-31,2012-08-25,708b81c9-21a9-411a-aae4-052ef8925b97,Levora 0.15/30 28 Day Pack
79,2014-08-15,2015-08-14,708b81c9-21a9-411a-aae4-052ef8925b97,Mirena 52 MG Intrauterine System
774,2010-04-05,2010-04-19,65b093e4-b353-447a-b9ae-290567f1c950,Acetaminophen 300 MG / HYDROcodone Bitartrate ...
775,2010-07-18,2010-08-29,65b093e4-b353-447a-b9ae-290567f1c950,Acetaminophen 300 MG / HYDROcodone Bitartrate ...
...,...,...,...,...
110215,1991-05-29,NaT,1e34d181-4497-4649-8fd3-b7e7f23109d2,NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Inje...
110216,2009-02-17,NaT,1e34d181-4497-4649-8fd3-b7e7f23109d2,Hydromorphone 325 MG
110217,2011-01-19,2011-01-27,1e34d181-4497-4649-8fd3-b7e7f23109d2,Penicillin V Potassium 500 MG Oral Tablet
110218,2017-12-06,2017-12-20,1e34d181-4497-4649-8fd3-b7e7f23109d2,Acetaminophen 325 MG Oral Tablet


From here, let's convert our medicine data frame into a "helper dictionary" that we can use when iterating through our encounters to set drug count and opioid indicators. Our medicine "helper" will group our patient's prescription history together by their unique patient ids. Here's how it will be structured:

`medict = {
            patient id: 
            {
                {
                "START": prescription start date,
                "STOP": prescription stop date,
                "DESCRIPTION": drug name
                }
                ...
            }
            ... 
           }`
           

In [32]:
medict = medicines.groupby('PATIENT')[
    'START', 'STOP', 'DESCRIPTION'].apply(
        lambda x: x.values.tolist()).to_dict()

Next, let's iterate through our encounters and check our medicine "helper" to determine if...
1. A medicine's start date < encounter start date.
2. A medicine's stop date >= encounter stop date.
3. A medicine's description contains something from our Opioid list.

In [33]:
valid_drugs_dict, opioid_dict = {}, {}
for row in encounters.itertuples():
    if medict.get(row.PATIENT_ID) is not None:
        for v in medict[row.PATIENT_ID]:
            # v = (Start, Stop, Description)
            if v[0] < row.HOSPITAL_ENCOUNTER_DATE and (
                    pd.isnull(v[1]) or v[1] >= row.STOP):
                if valid_drugs_dict.get(row.Index) is None:
                    valid_drugs_dict[row.Index] = 1
                else:
                    valid_drugs_dict[row.Index] += 1
                if ("Hydromorphone 325 MG" in v[2] or
                    "Oxycodone-acetaminophen 100ML" in v[2] or
                    "Fentanyl 100 MCG" in v[2]):
                    opioid_dict[row.Index] = 1

Things are getting a little more complicated, but we now have a couple more "helper" dictionaries which we can simply map to our encounters dataframe to build our `COUNT_CURRENT_MEDS` and `CURRENT_OPIOID_IND` columns.

In [35]:
encounters['COUNT_CURRENT_MEDS'] = encounters.index.map(valid_drugs_dict)
encounters['CURRENT_OPIOID_IND'] = encounters.index.map(opioid_dict)
encounters

,HOSPITAL_ENCOUNTER_DATE,STOP,PATIENT_ID,AGE_AT_VISIT,DEATH_AT_VISIT_IND,COUNT_CURRENT_MEDS,CURRENT_OPIOID_IND
ENCOUNTER_ID,,,,,,,
22874b3d-0873-40e4-a455-897d51d593f7,2012-02-18 21:50:51,2012-02-28 21:12:17,708b81c9-21a9-411a-aae4-052ef8925b97,25.339010,NaN,1.0,NaN
134c5ee3-1b72-4e3e-a44a-3efebe77603b,2013-08-03 21:50:51,2013-08-13 07:44:52,708b81c9-21a9-411a-aae4-052ef8925b97,26.794042,NaN,NaN,NaN
6125f147-72d4-48a4-9813-2a9dc878f055,2014-12-08 21:50:51,2014-12-17 12:25:27,708b81c9-21a9-411a-aae4-052ef8925b97,28.138888,NaN,1.0,NaN
f837dcf8-af7d-43b8-b5c2-773d494617ab,2015-08-31 21:50:51,2015-09-08 12:04:08,708b81c9-21a9-411a-aae4-052ef8925b97,28.864392,NaN,NaN,NaN
010594a6-a6ff-4875-9a9e-6b213367cde0,2018-12-10 00:59:35,2018-12-20 22:57:39,65b093e4-b353-447a-b9ae-290567f1c950,23.050321,NaN,NaN,NaN
...,...,...,...,...,...,...,...
09512352-7f84-4960-85be-9325aa91c023,2006-03-03 00:54:04,2006-03-08 06:12:14,12842850-ab85-4655-92e1-f8c5f1ac5cc2,24.294157,NaN,NaN,NaN
13459d81-c8e0-44ba-915f-b3cc7a32c118,2017-04-20 00:54:04,2017-04-30 09:28:08,12842850-ab85-4655-92e1-f8c5f1ac5cc2,35.440549,NaN,NaN,NaN
454afa58-c029-463e-8722-d795f1383150,2009-07-21 14:36:02,2009-07-29 02:22:25,5849b818-b6b3-4ed5-b3be-19d2f6fc2aa6,31.872246,NaN,1.0,1.0


Let's use this same "helper" technique to handle our indicators for `READMISSION_90_DAY_IND`, `READMISSION_30_DAY_IND`, and `FIRST_READMISSION_DATE`. This time we need to group encounters together by their patient's id and, since they are already sorted by date, check if the next encounter's start date (if applicable) is within 90/30 days of the current encounter's stop date. Like the previous step, let's store these values in a "helper" so that we can easily map their values to a new column in our encounters data frame.


In [36]:
# Prepare a data frame where encounters are grouped by patient ID
encounters.reset_index(inplace=True)
enc_grouped = encounters.groupby('PATIENT_ID')['ENCOUNTER_ID'].apply(lambda x: x.values)

# Build readmission indicator helpers
readd_dates, readd_30 = {}, {}
encounters.set_index('ENCOUNTER_ID', inplace=True)
for i in enc_grouped:
    if len(i) > 1:
        for j in range(len(i) - 1):
            next_diff = encounters.HOSPITAL_ENCOUNTER_DATE[i[j + 1]] - encounters.STOP[i[j]]
            if next_diff < np.timedelta64(91, 'D'):
                readd_dates[i[j]] = encounters.HOSPITAL_ENCOUNTER_DATE[i[j + 1]]
                if next_diff < np.timedelta64(31, 'D'):
                    readd_30[i[j]] = 1

And now we can map `readd_dates` and `readd_30` to our encounters data frame.

In [38]:
encounters['FIRST_READMISSION_DATE'] = encounters.index.map(readd_dates)
re_90 = ~pd.isnull(encounters['FIRST_READMISSION_DATE'].values)
encounters.loc[re_90, 'READMISSION_90_DAY_IND'] = 1
encounters['READMISSION_30_DAY_IND'] = encounters.index.map(readd_30)
encounters.reset_index(inplace=True)
encounters

,index,ENCOUNTER_ID,HOSPITAL_ENCOUNTER_DATE,STOP,PATIENT_ID,AGE_AT_VISIT,DEATH_AT_VISIT_IND,COUNT_CURRENT_MEDS,CURRENT_OPIOID_IND,FIRST_READMISSION_DATE,READMISSION_90_DAY_IND,READMISSION_30_DAY_IND
0,0,22874b3d-0873-40e4-a455-897d51d593f7,2012-02-18 21:50:51,2012-02-28 21:12:17,708b81c9-21a9-411a-aae4-052ef8925b97,25.339010,NaN,1.0,NaN,NaT,NaN,NaN
1,1,134c5ee3-1b72-4e3e-a44a-3efebe77603b,2013-08-03 21:50:51,2013-08-13 07:44:52,708b81c9-21a9-411a-aae4-052ef8925b97,26.794042,NaN,NaN,NaN,NaT,NaN,NaN
2,2,6125f147-72d4-48a4-9813-2a9dc878f055,2014-12-08 21:50:51,2014-12-17 12:25:27,708b81c9-21a9-411a-aae4-052ef8925b97,28.138888,NaN,1.0,NaN,NaT,NaN,NaN
3,3,f837dcf8-af7d-43b8-b5c2-773d494617ab,2015-08-31 21:50:51,2015-09-08 12:04:08,708b81c9-21a9-411a-aae4-052ef8925b97,28.864392,NaN,NaN,NaN,NaT,NaN,NaN
4,4,010594a6-a6ff-4875-9a9e-6b213367cde0,2018-12-10 00:59:35,2018-12-20 22:57:39,65b093e4-b353-447a-b9ae-290567f1c950,23.050321,NaN,NaN,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
429,429,09512352-7f84-4960-85be-9325aa91c023,2006-03-03 00:54:04,2006-03-08 06:12:14,12842850-ab85-4655-92e1-f8c5f1ac5cc2,24.294157,NaN,NaN,NaN,NaT,NaN,NaN
430,430,13459d81-c8e0-44ba-915f-b3cc7a32c118,2017-04-20 00:54:04,2017-04-30 09:28:08,12842850-ab85-4655-92e1-f8c5f1ac5cc2,35.440549,NaN,NaN,NaN,NaT,NaN,NaN
431,431,454afa58-c029-463e-8722-d795f1383150,2009-07-21 14:36:02,2009-07-29 02:22:25,5849b818-b6b3-4ed5-b3be-19d2f6fc2aa6,31.872246,NaN,1.0,1.0,NaT,NaN,NaN
432,432,29d0cb49-6a58-41e4-8ddd-d081eb0559ee,2018-08-03 22:14:33,2018-08-16 08:04:15,1e34d181-4497-4649-8fd3-b7e7f23109d2,28.390278,NaN,3.0,1.0,NaT,NaN,NaN


With those columns added, we can finally move on to part 3.
***
# Part 3: Export the data to a CSV file
Export a dataset containing these required fields:

| Field name                | Field Description.                                                                                                             | Data Type                                                                 |
|---------------------------|--------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------|
| `PATIENT_ID`              | Patient identifier.                                                                                                            | Character String                                                          |
| `ENCOUNTER_ID`            | Visit identifier.                                                                                                              | Character String                                                          |
| `HOSPITAL_ENCOUNTER_DATE` | Beginning of hospital encounter date.                                                                                          | Date/time                                                                 |
| `AGE_AT_VISIT`            | Patient age at admission.                                                                                                      | Num                                                                       |
| `DEATH_AT_VISIT_IND`      | Indicator if the patient died during the drug overdose encounter.                                                              | 0/1                         |
| `COUNT_CURRENT_MEDS`      | Count of active medications at the start of the drug overdose encounter.                                                       | Num                                                                       |
| `CURRENT_OPIOID_IND`      | Indicator if the patient had at least one active medication at the start of the overdose encounter that is on the Opioids List | 0/1                                                                       |
| `READMISSION_90_DAY_IND`  | Indicator if the visit resulted in a subsequent readmission within 90 days                                                     | 0/1                                                                       |
| `READMISSION_30_DAY_IND`  | Indicator if the visit resulted in a subsequent readmission within 30 days                                                     | 0/1                                                                       |
| `FIRST_READMISSION_DATE`  | Date of the first readmission for drug overdose within 90 days.                                                                | Date/time (Leave N/A if no readmissions for drug overdose within 90 days) |


***


We have all the required data in our encounters dataframe already, so let's prepare our data for export. This will take 3 steps:
1. Drop unneeded columns
2. Rearrange columns so their order matches the output requirements
3. Fill in NaN values as described in the output requirements

In [39]:
# Step 1: Drop unneeded columns
encounters.drop('STOP', axis=1, inplace=True)

# Step 2: Rearrange columns
encounters = encounters[(['PATIENT_ID',
                          'ENCOUNTER_ID',
                          'HOSPITAL_ENCOUNTER_DATE',
                          'AGE_AT_VISIT',
                          'DEATH_AT_VISIT_IND',
                          'COUNT_CURRENT_MEDS',
                          'CURRENT_OPIOID_IND',
                          'READMISSION_90_DAY_IND',
                          'READMISSION_30_DAY_IND',
                          'FIRST_READMISSION_DATE'])]

# Step 3: Fill in NA values
values = ({'COUNT_CURRENT_MEDS': 0,
           'CURRENT_OPIOID_IND': 0,
           'DEATH_AT_VISIT_IND': 0,
           'READMISSION_90_DAY_IND': 0,
           'READMISSION_30_DAY_IND': 0,
           'FIRST_READMISSION_DATE': 'NA'})
encounters = encounters.fillna(value=values)

Only one thing left to do: export our solution output.

In [41]:
encounters.to_csv(r'data_ex_pd_solution.csv', index=False)